In [1]:
import pandas as pd
import os
import numpy as np

def REINDEX(array):
    '''Sorts sentences by a given index in asencing order'''
    array_ascend = np.sort(array)
    unique, idx = [], []
    for i, x in enumerate(array_ascend):
        if x not in unique:
            idx.append(np.where(array==x)[0].tolist())
            unique.append(x)
    return sum(idx, [])

def main(df1, df2, exp_thresh=4.5, plaus_thresh=5, val_thresh=2, condition=None, disp=True, drop=True):
    df = pd.concat((df1, df2)).reset_index(drop=True);
    df = df.reindex(REINDEX(df['cloze(unexp_uncorrected)'])).reset_index(drop=True);
    
    num_zero_cloze = len([i for i in df['cloze(unexp_uncorrected)'].tolist() if i==0])
    new_df = pd.DataFrame(None, columns=df.columns)
    for i in range(len(df.head(num_zero_cloze))):
        if df['exp_mean'].iloc[i] <= exp_thresh: new_df.loc[len(new_df)] = df.iloc[i]     

    for i in range(len(df.head(num_zero_cloze)),len(df)):
        if df['Index'].iloc[i] in new_df['Index'].tolist(): new_df.loc[len(new_df)] = df.iloc[i]
    
    new_df = new_df.reindex(REINDEX(new_df['Index'])).reset_index(drop=True)
    
    new_df = new_df[new_df['plaus_mean']>=plaus_thresh]
    if condition == 'Emo':
        new_df = new_df[(new_df['S_val_mean']>=5+val_thresh) | (new_df['S_val_mean']<=5-val_thresh)].reset_index(drop=True)
    elif condition == 'Neu':
        new_df = new_df[(new_df['S_val_mean']<=5+val_thresh) & (new_df['S_val_mean']>=5-val_thresh)].reset_index(drop=True)
    new_df = new_df.reindex(REINDEX(new_df['Index'])).reset_index(drop=True)
    
    # drops sentences that do not have a counterpart
    if drop == True:
        idx = new_df['Index'].tolist()
        for i, x in enumerate(idx):
            if idx.count(x) == 1:
                new_df = new_df.drop(index=i)
                
    if drop == True:
        sc, wc = new_df[(new_df['Index']%4==0) | (new_df['Index']%4==1)], new_df[(new_df['Index']%4==2) | (new_df['Index']%4==3)]
        print('number of '+condition+'_SC frames (i.e., with counterparts) =', int(len(new_df[(new_df['Index']%4==0) | (new_df['Index']%4==1)])/2))
        if disp == True: display(new_df[(new_df['Index']%4==0) | (new_df['Index']%4==1)])
        print('number of '+condition+'_WC frames (i.e., with counterparts) =', int(len(new_df[(new_df['Index']%4==2) | (new_df['Index']%4==3)])/2))
        if disp == True: display(new_df[(new_df['Index']%4==2) | (new_df['Index']%4==3)])
    else:
        sc, wc = new_df[(new_df['Index']%4==0) | (new_df['Index']%4==1)], new_df[(new_df['Index']%4==2) | (new_df['Index']%4==3)]
        print('number of '+condition+'_SC sentences =', len(new_df[(new_df['Index']%4==0) | (new_df['Index']%4==1)]))
        if disp == True: display(new_df[(new_df['Index']%4==0) | (new_df['Index']%4==1)])
        print('number of '+condition+'_WC sentences =', len(new_df[(new_df['Index']%4==2) | (new_df['Index']%4==3)]))
        if disp == True: display(new_df[(new_df['Index']%4==2) | (new_df['Index']%4==3)])  
    
    return sc, wc

In [2]:
os.chdir('/mnt/c/Users/amand/Downloads/ratings_072323')
df1 = pd.read_excel('master_sheet_v3-1(2023.07.19_1st_round)_updated.xlsx', sheet_name='master_sheet').tail(487)
df2 = pd.read_excel('master_sheet_v3-2(2023.07.19_2nd_round).xlsx', sheet_name='master_sheet')

emo_sc, emo_wc = main(df1, df2, exp_thresh=8, plaus_thresh=3.7, val_thresh=1.6, condition='Emo', disp=False, drop=True);
neu_sc, neu_wc = main(df1, df2, exp_thresh=8, plaus_thresh=3.7, val_thresh=0.9, condition='Neu', disp=False, drop=True);

number of Emo_SC frames (i.e., with counterparts) = 82
number of Emo_WC frames (i.e., with counterparts) = 85
number of Neu_SC frames (i.e., with counterparts) = 77
number of Neu_WC frames (i.e., with counterparts) = 76


In [3]:
emo_sc_toSubset, emo_wc_toSubset = main(df1, df2, exp_thresh=8, plaus_thresh=3.7, val_thresh=1.6, condition='Emo', disp=False, drop=False);
neu_sc_toSubset, neu_wc_toSubset = main(df1, df2, exp_thresh=8, plaus_thresh=3.7, val_thresh=0.9, condition='Neu', disp=False, drop=False);

toSubset = [emo_sc_toSubset, emo_wc_toSubset, neu_sc_toSubset, neu_wc_toSubset]
subsetted = []
for f in toSubset:
    new_df = pd.DataFrame(None, columns=f.columns)
    idx = f['Index'].tolist()
    for i, x in enumerate(idx):
        if idx.count(x) == 1: new_df.loc[len(new_df)] = f.iloc[i]
    subsetted.append(new_df)
print('\n')    
to_print = ['number of singled out Emo_SC sentences (i.e., with no counterpart) = ', 'number of singled out Emo_WC sentences (i.e., with no counterpart) = ',
            'number of singled out Neu_SC sentences (i.e., with no counterpart) = ', 'number of singled out Neu_WC sentences (i.e., with no counterpart) = ']
for i, x in enumerate(subsetted):
    print(to_print[i] + str(len(x.index)))
    
disp, to_excel = False, False
if disp == True: dsipaly(x)
if to_excel == True: x.to_excel('/home/amandalin047/july_ratings/'+to_print[i]+'.xlsx', index=False)

number of Emo_SC sentences = 177
number of Emo_WC sentences = 187
number of Neu_SC sentences = 163
number of Neu_WC sentences = 163


number of singled out Emo_SC sentences (i.e., with no counterpart) = 13
number of singled out Emo_WC sentences (i.e., with no counterpart) = 17
number of singled out Neu_SC sentences (i.e., with no counterpart) = 9
number of singled out Neu_WC sentences (i.e., with no counterpart) = 11
